<a href="https://colab.research.google.com/github/rgrAndrade/case_predicao_fraude_cartao/blob/main/Detectar_Fraud_Cartao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
dados = pd.read_csv('https://raw.githubusercontent.com/rgrAndrade/case_predicao_fraude_cartao/refs/heads/main/Dados/fraud_dataset.csv')

Subir a base de dados e verificar si existe dados nulos

In [3]:
dados.head(2)

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0


In [4]:
dados.isnull().sum()

,0
distance_from_home,0
distance_from_last_transaction,0
ratio_to_median_purchase_price,0
repeat_retailer,0
used_chip,0
used_pin_number,1
online_order,1
fraud,1


In [5]:
dados.shape

(659835, 8)

Verificação de desiquilibrio de classe - fraud

In [6]:
round(dados['fraud'].value_counts(normalize=True)*100,2)

,proportion
fraud,
0.0,91.25
1.0,8.75


separação da base em treino e teste - técnica de houd out  (80% treino
 e 20% teste)

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

dados = dados.dropna(subset=['fraud'])

X = dados.drop('fraud', axis=1)
y = dados['fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=9)

In [8]:
print(X_train.shape, X_test.shape)

(527867, 7) (131967, 7)


Aplicando a padronização

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_escalonado = scaler.fit_transform(X_train)
X_test_escalonado = scaler.transform(X_test)



Aplicação do oversampling

In [18]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority', random_state=9)
X_train_resampled, y_train_resampled= smote.fit_resample(X_train_escalonado, y_train)

In [19]:
print(pd.Series(y_train_resampled).value_counts(normalize=True)*100)

fraud
0.0    50.0
1.0    50.0
Name: proportion, dtype: float64


Regressão Logística e Validação Cruzada


In [21]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

modelo = LogisticRegression(penalty='l2')

#validação cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
scores = cross_val_score(modelo, X_train_resampled, y_train_resampled, cv=cv, scoring= 'roc_auc')

In [23]:
print(scores.mean())

0.9794693521355551


In [24]:
modelo.fit(X_train_resampled, y_train_resampled)

y_pred = modelo.predict(X_test_escalonado)

In [26]:
from sklearn.metrics import classification_report, roc_auc_score

auc_score = roc_auc_score(y_test, y_pred)

print(classification_report(y_test, y_pred))

print(auc_score.mean())


              precision    recall  f1-score   support

         0.0       0.99      0.93      0.96    120424
         1.0       0.57      0.95      0.71     11543

    accuracy                           0.93    131967
   macro avg       0.78      0.94      0.84    131967
weighted avg       0.96      0.93      0.94    131967

0.9403844417776644
